In [1]:
print('hello')

hello


In [8]:
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
import numpy as np
import cv2 as cv2

In [4]:
model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)
model.eval()

SSD(
  (backbone): SSDLiteFeatureExtractorMobileNet(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (2): Hardswish()
        )
        (1): InvertedResidual(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            )
          )
        )
        (2): Invert

In [4]:
def preprocess_image(image):
    transform = T.Compose([
        T.Resize((300, 300)),
        T.ToTensor(),
    ])
    
    return transform(image)

In [5]:
def count_objects(predictions, threshold=0.3):
    scores = predictions[0]['scores'].detach().numpy()
    boxes = predictions[0]['boxes'].detach().numpy()
    labels = predictions[0]['labels'].detach().numpy()

    # Filter detections based on confidence threshold
    filtered_boxes = boxes[scores >= threshold]
    return len(filtered_boxes), filtered_boxes

In [6]:
def draw_boxes(frame, boxes):
    for box in boxes:
        # Unpack the coordinates
        x1, y1, x2, y2 = box
        # Draw a rectangle around the object
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

In [7]:
def detect_on_image(image_path):
    image = Image.open(image_path)
    input_tensor = preprocess_image(image).unsqueeze(0)  # Add batch dimension
    
    # Object detection
    with torch.no_grad():
        predictions = model(input_tensor)
    
    # Count objects and get bounding boxes
    num_objects, filtered_boxes = count_objects(predictions)

    # Convert image to array for display
    image_with_boxes = np.array(image)
    
    # Draw bounding boxes on the image
    draw_boxes(image_with_boxes, filtered_boxes)
    
    # Display results
    cv2.imshow("Object Detection on Image", cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(f"Total objects detected in image: {num_objects}")

In [8]:
# def detect_on_video(video_path=0):
#     # Capture from a video file or webcam (0 = default webcam)
#     cap = cv2.VideoCapture(video_path)

#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         if not ret:
#             break

#         # Convert frame (BGR) to PIL image (RGB) and preprocess
#         pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#         input_tensor = preprocess_image(pil_image).unsqueeze(0)

#         # Object detection
#         with torch.no_grad():
#             predictions = model(input_tensor)

#         # Count objects and get bounding boxes
#         num_objects, filtered_boxes = count_objects(predictions)

#         # Draw bounding boxes on the frame
#         draw_boxes(frame, filtered_boxes)

#         # Display the object count and the frame
#         cv2.putText(frame, f"Objects: {num_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#         cv2.imshow("Object Detection on Video", frame)

#         # Press 'q' to quit the video window
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [9]:
def detect_on_video(video_path=0):
    # Capture from a video file or webcam (0 = default webcam)
    cap = cv2.VideoCapture(video_path)

    total_objects_detected = 0  # Counter for total objects detected across the video

    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break

        # Convert frame (BGR) to PIL image (RGB) and preprocess
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        input_tensor = preprocess_image(pil_image).unsqueeze(0)

        # Object detection
        with torch.no_grad():
            predictions = model(input_tensor)

        # Count objects and get bounding boxes
        num_objects, filtered_boxes = count_objects(predictions)

        # Update the total object count
        total_objects_detected += num_objects

        # Draw bounding boxes on the frame
        draw_boxes(frame, filtered_boxes)

        # Display the object count and the frame
        cv2.putText(frame, f"Objects in frame: {num_objects}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Object Detection on Video", frame)

        # Press 'q' to quit the video window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When the video ends, print the final total object count
    print(f"Total objects detected in the video: {total_objects_detected}")

    cap.release()
    cv2.destroyAllWindows()


In [10]:
def main():
    # image_path = input("Enter the image path: ")
    # detect_on_image(image_path)
    video_path = input("Enter video file path or press Enter for webcam: ")
    if video_path == "":
        video_path = 0  # Default to webcam
    detect_on_image(video_path)

In [11]:
if __name__ == "__main__":
    main()

2024-10-08 12:15:53.463 python[2436:57967] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-08 12:15:53.463 python[2436:57967] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [7]:
import torch
import cv2
import numpy as np
import torchvision
from torchvision.models.detection import SSD300_VGG16_Weights, ssd300_vgg16
from torchvision.transforms import functional as F
from supervision import Detections
from PIL import Image

In [9]:
weights = SSD300_VGG16_Weights.COCO_V1  # You can also use SSD300_VGG16_Weights.DEFAULT
model = ssd300_vgg16(weights=weights)
model.eval()

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [10]:
coco_labels = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", 
    "boat", "traffic light", "fire hydrant", "N/A", "stop sign", "parking meter", "bench", "bird", 
    "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "N/A", "backpack", 
    "umbrella", "N/A", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", 
    "N/A", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", 
    "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", 
    "potted plant", "bed", "N/A", "dining table", "N/A", "toilet", "N/A", "tv", "laptop", "mouse", 
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "N/A", 
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# Create the mapping from class ID to label
model_labels = {i: label for i, label in enumerate(coco_labels)}

# def convert_ssd_to_supervision(results):
#     bboxes = []
#     confidences = []
#     class_ids = []

#     for i in range(len(results['boxes'])):
#         box = results['boxes'][i]
#         score = results['scores'][i]
#         label = results['labels'][i]

#         if score.item() > 0:  # Only consider detections with confidence > 0
#             bboxes.append(box.detach().cpu().numpy())
#             confidences.append(score.item())
#             class_ids.append(label.item())

#     bboxes = np.array(bboxes)
#     confidences = np.array(confidences)
#     class_ids = np.array(class_ids)

#     if len(bboxes) == 0:
#         return Detections(xyxy=np.empty((0, 4)), confidence=np.empty((0,)), class_id=np.empty((0,)))

#     detections = Detections(xyxy=bboxes, confidence=confidences, class_id=class_ids)
#     return detections

def detect_objects_in_frame_ssd(image, confidence_threshold=0.5):
    # Convert OpenCV image (BGR) to PIL image (RGB)
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Preprocess the image
    transform = T.Compose([
        T.Resize((300, 300)),  # SSD300 model input size
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for the SSD300 model
    ])
    image_tensor = transform(image_pil).unsqueeze(0)  # Add batch dimension
    
    # Perform object detection
    with torch.no_grad():
        detections = model(image_tensor)[0]  # Get the detections

    # Extract bounding boxes, labels, and confidence scores
    boxes = detections['boxes'].cpu().numpy()
    scores = detections['scores'].cpu().numpy()
    labels = detections['labels'].cpu().numpy()

    class_counts = {}  # To store object counts

    # Draw bounding boxes and count objects
    for i, box in enumerate(boxes):
        if scores[i] >= confidence_threshold:  # Only consider detections above confidence threshold
            class_id = labels[i]
            class_name = coco_labels[class_id]

            # Count the object
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

            # Draw bounding box
            x_min, y_min, x_max, y_max = map(int, box)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, f'{class_name}: {scores[i]:.2f}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    print(f"Detected objects: {class_counts}")  # Print the object counts in console

    # Create Detections object
    detection_objects = Detections(
        boxes=boxes[scores >= confidence_threshold],
        scores=scores[scores >= confidence_threshold],
        class_ids=labels[scores >= confidence_threshold]
    )

    return detection_objects  # Return the Detections object


       

In [11]:
def draw_boxes(frame, detections):
    for bbox, class_id in zip(detections.xyxy, detections.class_id):
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        label = coco_labels[class_id]  # Use class_id to get the class name
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

In [12]:
def detect_objects_in_frame_ssd(image, confidence_threshold=0.3):
    # Convert OpenCV image (BGR) to PIL image (RGB)
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Preprocess the image
    transform = T.Compose([
        T.Resize((300, 300)),  # SSD300 model input size
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for the SSD300 model
    ])
    image_tensor = transform(image_pil).unsqueeze(0)  # Add batch dimension
    
    # Perform object detection
    with torch.no_grad():
        detections = model(image_tensor)[0]  # Get the detections

    # Extract bounding boxes, labels, and confidence scores
    boxes = detections['boxes'].cpu().numpy()
    scores = detections['scores'].cpu().numpy()
    labels = detections['labels'].cpu().numpy()

    class_counts = {}  # To store object counts
    detected_objects = []

    # Draw bounding boxes and count objects
    for i, box in enumerate(boxes):
        if scores[i] >= confidence_threshold:  # Only consider detections above confidence threshold
            class_id = labels[i]
            class_name = coco_labels[class_id]

            # Count the object
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

            # Draw bounding box
            x_min, y_min, x_max, y_max = map(int, box)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, f'{class_name}: {scores[i]:.2f}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            detected_objects.append({
                'class_name': class_name,
                'confidence': scores[i],
                'box': (x_min, y_min, x_max, y_max)
            })

    print(f"Detected objects: {class_counts}")  # Print the object counts in console

    # Create a Detections object to return
    class Detections:
        def __init__(self, class_id, boxes):
            self.class_id = class_id
            self.boxes = boxes
    
    # Return the detections as a Detections object (this matches your existing `play` function expectations)
    return Detections(class_id=labels, boxes=boxes)

In [13]:
if __name__ == "__main__":
    image_path = '/Users/satyapraneeth/Desktop/cv-project/test_image_2.jpg'
    image = cv2.imread(image_path)
    detections = detect_objects_in_frame_ssd(image)  # Detect objects in the frame
    cv2.imshow("Detected Objects", image)  # Show the image with detections
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detected objects: {'person': 11, 'tie': 1}


2024-10-10 12:29:08.821 python[2511:72589] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-10 12:29:08.821 python[2511:72589] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
